# Projekt Sztucznej Sieci Neuronowej
## By Szymon Skrzypczyk

Jest to projekt prostej sieci neuronowej, która będzie się uczyć na funkcji liniowej(z możliwością zmiany, dzięki możliwości dodania wielu ukrytych warstw).

In [53]:
%use lets-plot
import kotlin.random.Random
import org.jetbrains.letsPlot.ggplot
import org.jetbrains.letsPlot.geom.geomLine
import org.jetbrains.letsPlot.label.ggtitle
import org.jetbrains.letsPlot.gggrid

### Definicja Neuronu

In [54]:
import kotlin.math.exp
import kotlin.random.Random

fun sigmoid(x: Double): Double {
    return 1.0 / (1.0 + exp(-x))
}

fun sumWeights(input: MutableList<Double>, weights: MutableList<Double>): Double {
    var sum = weights[0]
    for (i in 1..<input.size) {
        sum += input[i] * weights[i]
    }
    return sum
}


class Neuron(
    var input: MutableList<Double>,
    var activationFunction: (Double) -> Double = ::sigmoid,
) {
    var output: Double = 0.0
    var error: Double = 0.0
    var weights: MutableList<Double> = MutableList(input.size) { 0.0 }

    init {
        // Since it is a creation of a neuron, wights will have random values
        for (i in weights.indices) {
            weights[i] = Random.nextDouble(-1.0, 1.0)
        }
    }

    fun feedForward() {
        val weightedSum = sumWeights(input, weights)
        output = activationFunction(weightedSum)
    }

    fun calculateError(target: Double) {
        error = target - output
    }

    fun updateWeights(learningRate: Double) {
        for (i in weights.indices) {
            weights[i] += learningRate * error * input[i]
        }
    }
}

## Definicja Sieci Neuronowej

In [55]:
class NeuralNetwork(
    private val inputSize: Int,
    private val hiddenLayers: List<Int>,
    private val outputSize: Int,
    private val activationFunction: (Double) -> Double = ::sigmoid
) {
    private val layers: MutableList<MutableList<Neuron>> = mutableListOf()

    init {
        var previousSize = inputSize
        for (layerSize in hiddenLayers) {
            val layer = MutableList(layerSize) {
                Neuron(MutableList(previousSize) { 0.0 }, activationFunction)
            }
            layers.add(layer)
            previousSize = layerSize
        }
        val outputLayer = MutableList(outputSize) {
            Neuron(MutableList(previousSize) { 0.0 }, activationFunction)
        }
        layers.add(outputLayer)
    }

    fun feedForward(input: MutableList<Double>): MutableList<Double> {
        var currentInput = input
        for (layer in layers) {
            for (neuron in layer) {
                neuron.input = currentInput
                neuron.feedForward()
            }
            currentInput = layer.map { it.output }.toMutableList()
        }
        return currentInput
    }

    fun backpropagate(target: MutableList<Double>, learningRate: Double) {
        val outputLayer = layers.last()

        for (i in outputLayer.indices) {
            outputLayer[i].calculateError(target[i])
        }

        for (i in layers.size - 2 downTo 0) {
            val currentLayer = layers[i]
            val nextLayer = layers[i + 1]

            for (j in currentLayer.indices) {
                val neuron = currentLayer[j]
                val sum = nextLayer.sumOf { nextNeuron ->
                    if (j + 1 < nextNeuron.weights.size) {
                        nextNeuron.error * nextNeuron.weights[j + 1]
                    } else 0.0
                }
                neuron.error = sum * neuron.output * (1 - neuron.output)
            }
        }

        for (layer in layers) {
            for (neuron in layer) {
                neuron.updateWeights(learningRate)
            }
        }
    }

    fun train(inputs: List<MutableList<Double>>, targets: List<MutableList<Double>>, epochs: Int, learningRate: Double) {
        for (epoch in 0 until epochs) {
            for (i in inputs.indices) {
                feedForward(inputs[i])
                backpropagate(targets[i], learningRate)
            }
        }
    }

    fun predict(input: MutableList<Double>): MutableList<Double> {
        return feedForward(input)
    }
}

In [56]:
object Utils {
    fun MeanSquaredError(target: MutableList<Double>, predicted: MutableList<Double>): Double {
        var sum = 0.0
        for (i in target.indices) {
            sum += (target[i] - predicted[i]).pow(2)
        }
        return sum / target.size
    }
}

### Zdefiniowanie funkcji do uczenia

In [57]:
val step = 12.0 / 499
val x = generateSequence(0.0) { it + step }
    .takeWhile { it <= 12 }
    .toMutableList()
val y = x.map { 0.5 * cos(0.2 * it * it) + 0.5}.toMutableList()


In [58]:
val data = mapOf<String, Any>(
    "x" to x,
    "y" to y
)

val plot = ggplot(data) +
        geomLine(color = "red") { x = "x"; y = "y" } +
        ggtitle("y = 0.5 * cos(0.2 * x²) + 0.5")

plot

<path d="M24.407513580896307 14.590909090909122 L24.407513580896307 14.590909090909122 L25.38577063824687 14.590910066880554 L26.364027695597436 14.590924706451233 L27.342284752948 14.590988144585424 L28.320541810298565 14.59115893951622 L29.29879886764913 14.59151907260923 L30.277055924999694 14.592173948016352 L31.255312982350258 14.593252391968747 L32.23357003970082 14.594906651510371 L33.21182709705138 14.59731239243007 L34.19008415440195 14.600668696109153 L35.16834121175251 14.605198054953291 L36.14659826910308 14.611146366037417 L37.12485532645364 14.618782922546643 L38.10311238380421 14.628400402552245 L39.08136944115478 14.64031485461976 L40.05962649850534 14.654865679700094 L41.037883555855906 14.672415608713663 L42.016140613206474 14.69335067519188 L42.994397670557035 14.718080182297513 L43.972654727907596 14.747036663503764 L44.950911785258164 14.780675836166324 L45.92916884260873 14.81947654718158 L46.90742589995929 14.86394070988132 L47.885682957309854 14.91459323127134 L48.86394001466043 14.971981928679668 L49.84219707201099 15.036677434839248 L50.82045412936155 15.10927309038766 L51.79871118671212 15.190384822726287 L52.77696824406269 15.280651010141582 L53.75522530141325 15.38073233005133 L54.733482358763816 15.491311590200098 L55.71173941611438 15.613093541589762 L56.689996473464944 15.746804671895461 L57.66825353081551 15.893192978079242 L58.64651058816607 16.053027716879797 L59.62476764551664 16.22709913182382 L60.60302470286721 16.41621815537019 L61.58128176021777 16.621216084768093 L62.55953881756834 16.842944230182013 L63.537795874918906 17.082273533605814 L64.51605293226947 17.34009415706555 L65.49430998962004 17.61731503858499 L66.4725670469706 17.91486341436513 L67.45082410432116 18.23368430561254 L68.42908116167172 18.574739968431686 L69.4073382190223 18.939009305183276 L70.38559527637285 19.327487235699152 L71.36385233372343 19.741184026736335 L72.34210939107399 20.181124578046365 L73.32036644842455 20.64834766343654 L74.29862350577511 21.143905125199694 L75.27688056312567 21.66886102029582 L76.25513762047625 22.22429071667989 L77.23339467782681 22.811279938182565 L78.21165173517738 23.430923756371556 L79.18990879252794 24.084325527843816 L80.1681658498785 24.772595775428385 L81.14642290722907 25.496851011813646 L82.12467996457963 26.258212504152425 L83.1029370219302 27.057804978245372 L84.08119407928078 27.896755260952318 L85.05945113663134 28.776190859542623 L86.0377081939819 29.697238476757946 L87.01596525133246 30.661022460433855 L87.99422230868302 31.668663186605727 L88.9724793660336 32.72127537511 L89.95073642338416 33.819966336787274 L90.92899348073472 34.96583415149445 L91.9072505380853 36.159965776244974 L92.88550759543585 37.4034350829138 L93.86376465278641 38.697300825072546 L94.84202171013699 40.04260453365572 L95.82027876748755 41.44036834130566 L96.79853582483811 42.8915927353986 L97.77679288218869 44.39725423991894 L98.75504993953925 45.95830302652428 L99.73330699688981 47.575660455326954 L100.71156405424038 49.250216546114245 L101.68982111159094 50.98282738093383 L102.6680781689415 52.77431243918491 L103.64633522629207 54.62545186658531 L104.62459228364264 56.53698367961684 L105.6028493409932 58.509600907301376 L106.58110639834376 60.543948672416974 L107.55936345569434 62.64062121453074 L108.5376205130449 64.80015885750618 L109.51587757039546 67.02304492442926 L110.49413462774604 69.3097026031991 L111.4723916850966 71.66049176633828 L112.45064874244716 74.0757057488971 L113.42890579979773 76.55556808865589 L114.4071628571483 79.1002292331674 L115.38541991449885 81.70976321853004 L116.36367697184942 84.38416432513773 L117.34193402919999 87.12334371601685 L118.32019108655055 89.92712606373337 L119.29844814390111 92.79524617223001 L120.27670520125169 95.72734560034095 L121.25496225860225 98.72296929411974 L122.23321931595281 101.78156223551332 L123.21147637330338 104.90246611531364 L124.18973343065394 108.08491603872005 L125.1679904880045 111.32803727225158 L126.14624754535507 114.63084

### Zdefiniowanie zmiennych

In [59]:
val ileDanych = 500
val ileEpok = 100

In [60]:
val x_siec = MutableList(ileDanych) {
    12 * Random.nextDouble()
}

val y_siec: MutableList<Double> = x_siec.map {
    0.5 * cos(0.2 * it * it) + 0.5 + 0.1 * Random.nextDouble(-1.0, 1.0)
}.toMutableList()

In [61]:
val data = mapOf<String, Any>(
    "x" to x_siec,
    "y" to y_siec
)

val plot = ggplot(data) +
        geomLine(color = "red") { x = "x"; y = "y" } +
        ggtitle("y = 0.5 * cos(0.2 * x²) + 0.5 with noise")

plot

<path d="M24.407513580896303 30.245452005331686 L24.407513580896303 30.245452005331686 L24.632380867360904 22.268286448671404 L25.23239473075991 44.2053436940453 L25.70461417077394 28.060906033375716 L26.081294822443024 41.42485459283051 L26.285310282380077 17.65734163163296 L26.683231413040325 46.00118837872975 L26.822235468059123 27.79315073981479 L28.12679083880606 24.503704357301444 L28.77828075790058 61.39730700735933 L30.60698574327604 25.039643268126042 L32.819205842664985 29.796529446212503 L32.90546752405348 31.944398410992846 L33.5551528477519 22.59964492865123 L37.35168438207411 60.34194383190288 L39.13605534076738 58.96204483227402 L40.7885905461746 45.28322350887322 L40.99224653228277 24.3034606591267 L42.49737072236242 48.0147578760066 L42.68421609083504 16.62677364274981 L47.720541056395305 16.616442372830875 L47.92046669904298 26.634483258080877 L48.49542534327401 30.603897175797215 L50.52282872939146 45.992687743371306 L51.233129713371355 35.69829646748482 L51.25816985334757 28.477838462769682 L51.823218629122145 53.99196752414204 L51.94705580994727 17.311719762717587 L52.3880091169702 39.273251335305076 L52.408057697467825 50.12437468060193 L52.65206640274815 27.24609336009982 L53.10524881811262 25.963495449092022 L55.683089981612675 29.019677360759545 L56.83064695141309 15.390690780024386 L57.41013317561956 27.750296821917033 L60.13209016613041 41.22691221197067 L60.38651030609658 24.267915040523405 L60.95260948455882 45.02061074331448 L61.03212026301746 27.497532162181784 L61.04556824569012 63.49054632057147 L61.115201908333376 23.219522159002167 L63.20876805582816 59.35366229055933 L63.470153335698214 55.3192528661171 L63.9951816552599 22.440839464981934 L64.27570180823896 63.42585792196249 L64.5850760841265 43.0227569224287 L67.92333959414123 19.98019383547131 L68.4044309359559 18.731129009417486 L68.73170190053932 25.130001824003784 L69.19699234861893 57.302019295981324 L70.11680453797717 48.64449650620753 L70.38979549763154 33.17415383110884 L70.54618868581956 41.82219536703013 L71.61920748504335 44.02866628636099 L72.82904629069036 31.737228490752727 L74.08683391995226 65.0611447648256 L74.70902924025118 30.797007572060096 L75.4449364218233 26.14479959810666 L75.98768347438636 25.295969388591118 L76.908865123621 32.15421021214189 L77.70030105011888 50.53411351764245 L77.88146358691168 29.850875681493164 L78.75696709136855 59.344325273700406 L78.85642454845188 68.04222853862788 L79.85290066378329 52.88387940801067 L79.92848765081173 36.43595380610182 L80.10289288008515 63.01400320149361 L81.64488195899017 43.16511617304715 L81.87998361605264 67.14225337279512 L82.07292501746372 52.80965485982941 L82.94651577523192 47.63774896279841 L85.09248487460461 45.156610750541034 L85.76344516475334 26.171736655843176 L86.2370333360976 48.51093704089979 L86.54038593449259 45.391119524558235 L88.38265531631485 54.488388097592036 L90.72413201695356 58.67342977482673 L90.96338167135063 41.43332536644701 L91.17870217624707 43.15419606518884 L91.96470893903535 38.52868824975516 L92.50830825625448 79.80562479824661 L92.93757331023035 73.81394196313587 L93.79791853066317 62.82396005919466 L95.07263584049639 76.57923860711699 L95.741803566109 83.85887279029731 L96.90125653776167 54.740856258908934 L97.98198413340114 78.23145925658048 L99.56796726705866 43.07137094589612 L100.78904457218722 63.25255094405935 L101.14554701359651 58.77449674998988 L101.77689911836072 68.21503457894582 L101.8923106034283 63.377302451343894 L103.33971797309991 83.23759333898886 L103.79995875031635 51.015174082824814 L104.14852634510451 68.16486861205152 L105.35744721333243 67.43821370950269 L105.45564152598321 68.3687536721387 L105.50295581664314 88.57921059148694 L105.65403489711554 57.17787340872346 L106.59354335357443 63.457422088756914 L106.66995396517406 72.18077310977512 L108.37475862116064 75.39489226933233 L109.57252445629582 61.509533181363906 L109.9276867327534 105.51299448202107 L110.89106347020645 86.51705505981616 L111.472131473203

### Nałożenie dwóch wykresów

In [62]:
val data_siec = mapOf<String, Any>(
    "x_siec" to x_siec,
    "y_siec" to y_siec
)

val data_clean = mapOf<String, Any>(
    "x" to x,
    "y" to y
)

val plot = ggplot() +
        geomLine(data = data_siec, color = "red") { x = "x_siec"; y = "y_siec" } +
        geomLine(data = data_clean, color = "green") { x = "x"; y = "y" } +
        ggtitle("Nałożenie wykresów")

plot

<path d="M24.434590515139373 30.245452005331686 L24.434590515139373 30.245452005331686 L24.65923135833369 22.268286448671404 L25.258641002668305 44.2053436940453 L25.730384913689363 28.060906033375716 L26.106686244738025 41.42485459283051 L26.310496259371487 17.65734163163296 L26.70801668006829 46.00118837872975 L26.846880756821264 27.79315073981479 L28.150122429212804 24.503704357301444 L28.80095629241697 61.39730700735933 L30.62781975631649 25.039643268126042 L32.83781213124735 29.796529446212503 L32.92398694638891 31.944398410992846 L33.573018031439894 22.59964492865123 L37.365726426212284 60.34194383190288 L39.1483005081716 58.96204483227402 L40.79917159656987 45.28322350887322 L41.002622499367654 24.3034606591267 L42.50623101658473 48.0147578760066 L42.69288823018191 16.62677364274981 L47.724141573660944 16.616442372830875 L47.923865889485825 26.634483258080877 L48.49824554547129 30.603897175797215 L50.52360731913305 45.992687743371306 L51.23319302397998 35.69829646748482 L51.25820794832224 28.477838462769682 L51.82268771517309 53.99196752414204 L51.94640019090372 17.311719762717587 L52.38690945419556 39.273251335305076 L52.40693784560207 50.12437468060193 L52.65070083204054 27.24609336009982 L53.103426888857804 25.963495449092022 L55.6786721443781 29.019677360759545 L56.82507351454836 15.390690780024386 L57.403976191197735 27.750296821917033 L60.123192147865126 41.22691221197067 L60.37735608458614 24.267915040523405 L60.942885196360216 45.02061074331448 L61.02231590678859 27.497532162181784 L61.03575034722832 63.49054632057147 L61.10531388818096 23.219522159002167 L63.19677179675653 59.35366229055933 L63.45789385942631 55.3192528661171 L63.98239347100411 22.440839464981934 L64.26263113780291 63.42585792196249 L64.57169387116298 43.0227569224287 L67.90659572142656 19.98019383547131 L68.38720260016768 18.731129009417486 L68.71414400010588 25.130001824003784 L69.17896589674632 57.302019295981324 L70.0978518274056 48.64449650620753 L70.37056788284143 33.17415383110884 L70.5268035817586 41.82219536703013 L71.59874184192405 44.02866628636099 L72.80736232960243 31.737228490752727 L74.06388335602242 65.0611447648256 L74.68545212034118 30.797007572060096 L75.42061823712154 26.14479959810666 L75.96281873878586 25.295969388591118 L76.88307275026379 32.15421021214189 L77.67371169405143 50.53411351764245 L77.85469179862864 29.850875681493164 L78.72931366360916 59.344325273700406 L78.82867096618718 68.04222853862788 L79.82414362159405 52.88387940801067 L79.89965449188358 36.43595380610182 L80.07388409360755 63.01400320149361 L81.61432037639335 43.16511617304715 L81.8491852840873 67.14225337279512 L82.04193239186591 52.80965485982941 L82.91464343630996 47.63774896279841 L85.05845152654514 45.156610750541034 L85.72873615397424 26.171736655843176 L86.20184741800142 48.51093704089979 L86.50489453774941 45.391119524558235 L88.34530873863312 54.488388097592036 L90.68442755231678 58.67342977482673 L90.92343628027672 41.43332536644701 L91.13853995559313 43.15419606518884 L91.92375520288519 38.52868824975516 L92.46680711096782 79.80562479824661 L92.89563989138284 73.81394196313587 L93.75511873686027 62.82396005919466 L95.02855239551924 76.57923860711699 L95.69704626353997 83.85887279029731 L96.85533165618742 54.740856258908934 L97.934970949945 78.23145925658048 L99.51935698509595 43.07137094589612 L100.73920465499128 63.25255094405935 L101.09534809540953 58.77449674998988 L101.72606442323371 68.21503457894582 L101.8413596879541 63.377302451343894 L103.28730950609224 83.23759333898886 L103.74708681693104 51.015174082824814 L104.0953034011862 68.16486861205152 L105.30300687581628 67.43821370950269 L105.40110230595893 68.3687536721387 L105.4483689507257 88.57921059148694 L105.59929589327892 57.17787340872346 L106.53785825673033 63.457422088756914 L106.61419192219411 72.18077310977512 L108.31727982539039 75.39489226933233 L109.51383950014092 61.509533181363906 L109.86864412516496 105.51299448202107 L110.83105073404853 86.51705505981616 L111.411533

In [63]:
val nn = NeuralNetwork(
    inputSize = 500,
    hiddenLayers = listOf(5),
    outputSize = 500
)

In [64]:
nn.train(
    inputs = listOf(x_siec),
    targets = listOf(y_siec),
    epochs = ileEpok,
    learningRate = 0.01
)

val y_pred = nn.predict(x_siec)

In [65]:
val data_siec = mapOf<String, Any>(
    "x_siec" to x_siec,
    "y_siec" to y_siec
)

val data_pred = mapOf<String, Any>(
    "x" to x_siec,
    "y" to y_pred
)

val plot = ggplot() +
        geomLine(data = data_siec, color = "red") { x = "x_siec"; y = "y_siec" } +
        geomLine(data = data_pred, color = "green") { x = "x"; y = "y" } +
        ggtitle("Nałożenie wykresów")

plot

<path d="M24.407513580896303 30.245452005331686 L24.407513580896303 30.245452005331686 L24.632380867360904 22.268286448671404 L25.23239473075991 44.2053436940453 L25.70461417077394 28.060906033375716 L26.081294822443024 41.42485459283051 L26.285310282380077 17.65734163163296 L26.683231413040325 46.00118837872975 L26.822235468059123 27.79315073981479 L28.12679083880606 24.503704357301444 L28.77828075790058 61.39730700735933 L30.60698574327604 25.039643268126042 L32.819205842664985 29.796529446212503 L32.90546752405348 31.944398410992846 L33.5551528477519 22.59964492865123 L37.35168438207411 60.34194383190288 L39.13605534076738 58.96204483227402 L40.7885905461746 45.28322350887322 L40.99224653228277 24.3034606591267 L42.49737072236242 48.0147578760066 L42.68421609083504 16.62677364274981 L47.720541056395305 16.616442372830875 L47.92046669904298 26.634483258080877 L48.49542534327401 30.603897175797215 L50.52282872939146 45.992687743371306 L51.233129713371355 35.69829646748482 L51.25816985334757 28.477838462769682 L51.823218629122145 53.99196752414204 L51.94705580994727 17.311719762717587 L52.3880091169702 39.273251335305076 L52.408057697467825 50.12437468060193 L52.65206640274815 27.24609336009982 L53.10524881811262 25.963495449092022 L55.683089981612675 29.019677360759545 L56.83064695141309 15.390690780024386 L57.41013317561956 27.750296821917033 L60.13209016613041 41.22691221197067 L60.38651030609658 24.267915040523405 L60.95260948455882 45.02061074331448 L61.03212026301746 27.497532162181784 L61.04556824569012 63.49054632057147 L61.115201908333376 23.219522159002167 L63.20876805582816 59.35366229055933 L63.470153335698214 55.3192528661171 L63.9951816552599 22.440839464981934 L64.27570180823896 63.42585792196249 L64.5850760841265 43.0227569224287 L67.92333959414123 19.98019383547131 L68.4044309359559 18.731129009417486 L68.73170190053932 25.130001824003784 L69.19699234861893 57.302019295981324 L70.11680453797717 48.64449650620753 L70.38979549763154 33.17415383110884 L70.54618868581956 41.82219536703013 L71.61920748504335 44.02866628636099 L72.82904629069036 31.737228490752727 L74.08683391995226 65.0611447648256 L74.70902924025118 30.797007572060096 L75.4449364218233 26.14479959810666 L75.98768347438636 25.295969388591118 L76.908865123621 32.15421021214189 L77.70030105011888 50.53411351764245 L77.88146358691168 29.850875681493164 L78.75696709136855 59.344325273700406 L78.85642454845188 68.04222853862788 L79.85290066378329 52.88387940801067 L79.92848765081173 36.43595380610182 L80.10289288008515 63.01400320149361 L81.64488195899017 43.16511617304715 L81.87998361605264 67.14225337279512 L82.07292501746372 52.80965485982941 L82.94651577523192 47.63774896279841 L85.09248487460461 45.156610750541034 L85.76344516475334 26.171736655843176 L86.2370333360976 48.51093704089979 L86.54038593449259 45.391119524558235 L88.38265531631485 54.488388097592036 L90.72413201695356 58.67342977482673 L90.96338167135063 41.43332536644701 L91.17870217624707 43.15419606518884 L91.96470893903535 38.52868824975516 L92.50830825625448 79.80562479824661 L92.93757331023035 73.81394196313587 L93.79791853066317 62.82396005919466 L95.07263584049639 76.57923860711699 L95.741803566109 83.85887279029731 L96.90125653776167 54.740856258908934 L97.98198413340114 78.23145925658048 L99.56796726705866 43.07137094589612 L100.78904457218722 63.25255094405935 L101.14554701359651 58.77449674998988 L101.77689911836072 68.21503457894582 L101.8923106034283 63.377302451343894 L103.33971797309991 83.23759333898886 L103.79995875031635 51.015174082824814 L104.14852634510451 68.16486861205152 L105.35744721333243 67.43821370950269 L105.45564152598321 68.3687536721387 L105.50295581664314 88.57921059148694 L105.65403489711554 57.17787340872346 L106.59354335357443 63.457422088756914 L106.66995396517406 72.18077310977512 L108.37475862116064 75.39489226933233 L109.57252445629582 61.509533181363906 L109.9276867327534 105.51299448202107 L110.89106347020645 86.51705505981616 L111.472131473203

Jak widać ogólny kształt wykresu jest zachowany, ale nie jest on idealny, a właściwie nie jest nawet bliski idealnemu - w kolejnym kroku porównanie dla różnych ilości epok i warstw

In [73]:
import org.jetbrains.letsPlot.core.spec.plotson.plot

val params: Map<String, List<Any>> = mapOf(
    "epoki" to listOf(10, 50, 100, 200, 250, 500),
    "warstwy" to listOf(5, 4, 2, 1, 5, 1),
    "learningRate" to listOf(0.1, 0.01, 0.1, 0.25, 0.15, 0.1),
)

val plots: MutableList<Figure?> = mutableListOf()

for (current_iteration in 0 until params["epoki"]!!.size) {
    val current_epochs = params["epoki"]!![current_iteration] as Int
    val current_layers = params["warstwy"]!![current_iteration] as Int
    val current_learningRate = params["learningRate"]!![current_iteration] as Double

    val nn = NeuralNetwork(
        inputSize = 500,
        hiddenLayers = List(current_layers) { 5 },
        outputSize = 500
    )

    nn.train(
        inputs = listOf(x_siec),
        targets = listOf(y_siec),
        epochs = current_epochs,
        learningRate = current_learningRate
    )

    val y_pred = nn.predict(x)

    val data_siec = mapOf<String, Any>(
        "x_siec" to x_siec,
        "y_siec" to y_siec
    )

    val data_pred = mapOf<String, Any>(
        "x" to x_siec,
        "y" to y_pred
    )

    val plot = ggplot() +
            geomLine(data = data_siec, color = "red") { x = "x_siec"; y = "y_siec" } +
            geomLine(data = data_pred, color = "green") { x = "x"; y = "y" } +
            ggtitle("epoki: $current_epochs, warstwy: $current_layers, lr: $current_learningRate")

    plots.add(plot)
}

gggrid(plots, ncol = 3).show()